In [ ]:
# --> 37
array = [1, 2, [3, 4, (5), 6, {7}, 8], 1] 

In [2]:
def recursive_sum(array):
    global sm 
    iterable = []
    for el in array:
        if hasattr(el, '__iter__'):
            iterable.append(el)
        elif type(el) in [int, float]:
            sm += el

    for el in iterable:
        recursive_sum(el)
    
array = [1, 2, [3, 4, (5), 6, {7}, 8], 1] * 1000 
sm = 0
recursive_sum(array)     
sm 

37000

In [1]:
def recursive_sum(*args):
    total_sum = 0 
    for arg in args:
        if hasattr(arg, '__iter__') and not isinstance(arg, str):
            total_sum += recursive_sum(*arg) 
        elif isinstance(arg, (int, float)):
            total_sum += arg  
    return total_sum 

array = [1, 2, 'abc', [3, 4, (5), 6, {7, 'xyz'}, 8], 1] * 1000 
recursive_sum(array) 

37000

In [ ]:
'''
Строка 'abc' на позиции [2]
Строка 'xyz' на позиции [3][4][0]
'''

In [8]:
%%time 

def recursive_sum(*args, path=[]):
    total_sum = 0 
    for index, arg in enumerate(args):
        if hasattr(arg, '__iter__') and not isinstance(arg, str):
            total_sum += recursive_sum(*arg, path=path+[index]) 
        elif isinstance(arg, (int, float)):
            total_sum += arg  
        # elif isinstance(arg, str):
        #     print(f"Строка '{arg}' на позиции", ''.join(f'[{n}]' for n in path+[index]))
    return total_sum 

array = [1, 2, 'abc', [3, 4, (5), 6, {7, 'xyz'}, 8], 1] * 1000 
recursive_sum(*array) 

CPU times: total: 15.6 ms
Wall time: 9 ms


37000

In [10]:
%%time 

import asyncio
import nest_asyncio
nest_asyncio.apply()

async def recursive_sum(*args, path=[]):
    total_sum = 0 
    for index, arg in enumerate(args):
        if hasattr(arg, '__iter__') and not isinstance(arg, str):
            total_sum += await recursive_sum(*arg, path=path+[index]) 
        elif isinstance(arg, (int, float)):
            total_sum += arg  
        # elif isinstance(arg, str):
        #     print(f"Строка '{arg}' на позиции", ''.join(f'[{n}]' for n in path+[index]))
    return total_sum 

array = [1, 2, 'abc', [3, 4, (5), 6, {7, 'xyz'}, 8], 1] * 1000 

loop = asyncio.get_event_loop()
result = loop.run_until_complete(recursive_sum(*array))
print(result)

37000
CPU times: total: 31.2 ms
Wall time: 23 ms


In [11]:
%%time 
import concurrent.futures

def recursive_sum(*args, path=[]):
    total_sum = 0 
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for index, arg in enumerate(args):
            if hasattr(arg, '__iter__') and not isinstance(arg, str):
                future = executor.submit(recursive_sum, *arg, path=path+[index])
                futures.append(future)
            elif isinstance(arg, (int, float)):
                total_sum += arg  
            # elif isinstance(arg, str):
            #     print(f"Строка '{arg}' на позиции", ''.join(f'[{n}]' for n in path[1:]+[index]))
        for future in concurrent.futures.as_completed(futures):
            total_sum += future.result()
    return total_sum

b = [1, 2, 'abc', [3, 4, (5), 6, {7, 'xyz'}, 8], 1] * 1000
result = recursive_sum(b)
print(result)

37000
CPU times: total: 672 ms
Wall time: 632 ms
